<a href="https://colab.research.google.com/github/buwuyou/CTCN_2020_EE/blob/master/CTCN_soil_salinity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook shows how to apply remote sensing techniques on soil salinity mapping, including:
1. Long-term Landsat time series preparation
2. Calculate Enhanced Salinity Indice
3. Apply trend analysis for the period of 1988-2019
4. Export the product

Prerequisites: CTCN 101 and CTCN time series analysis

### Install, Import, Authenticate and Initialize EE Python API

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [0]:
!pip install -q earthengine-api

# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()

In [0]:
from pprint import pprint 

import folium
from folium import plugins
print(folium.__version__)
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Prepare long-term Landsat time series
To understand the code of this session, some essential knowledge on Landsat archive is required, such as the different spectrometers mounted on the satellite. Specifically, since Landsat 8 era, new version of radiometer has been used causing slightly spectral difference in radiation measurements between the old Thematic Mapper radiometers and Operational Land Imager radiometers. To combined the observations, a calibration model is required, which is about the function of **harmonizationRoy**

Furthermore, it is a common technique to make a composite image from several images during a certain period to get a cloud-free image. There are sevaral techniques to do so, such as mean, median, and mediod. The adopted method as seasonal mediod composite is optimal for mainting the spectral relationship amongst bands by calcuating an artificial mediod value using the avaialble observations. Check here to understand better: https://www.mdpi.com/2072-4292/5/12/6481 

To prepare the long-term Landsat time series, we did it via LandTrendr. [LandTrendr](https://emapr.github.io/LT-GEE/introduction.html) is a set of spectral-temporal segmentation algorithms that are useful for change detection in a time series of moderate resolution satellite imagery (primarily Landsat) and for generating trajectory-based spectral time series data largely absent of inter-annual signal noise. We implement the Landsat annual data preparation part of LandTrendr Earth Engine JavaScript framework into the Earth Engine Python API.

>Kennedy, R.E., Yang, Z., Gorelick, N., Braaten, J., Cavalcante, L., Cohen, W.B., Healey, S. (2018). Implementation of the LandTrendr Algorithm on Google Earth Engine. Remote Sensing. 10, 691.

### Import functions

In [0]:
# rewrite LandTrendr EE Javascript to EE Python:

# Cloud masking function.
def maskLsr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int() # cloud shadow
  cloudsBitMask = ee.Number(2).pow(5).int() # clouds
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

#------ L8 to L7 HARMONIZATION FUNCTION -----
#slope and intercept citation: Roy, D.P., Kovalskyy, V., Zhgang, H.K., Vermote, E.F., Yan, L., Kumar, S.S, Egorov, A., 2016, Characterization of Landsat-7 to Landsat-8 reflective wavelength and normalized difference vegetation index continuity, Remote Sensing of Environment, 185, 57-70.(http://dx.doi.org/10.1016/j.rse.2015.12.024); Table 2 - reduced major axis (RMA) regression coefficients
def harmonizationRoy(oli):
  slopes = ee.Image.constant([0.9785, 0.9542, 0.9825, 1.0073, 1.0171, 0.9949])        # RMA - create an image of slopes per band for L8 TO L7 regression line - David Roy
  itcp = ee.Image.constant([-0.0095, -0.0016, -0.0022, -0.0021, -0.0030, 0.0029])     # RMA - create an image of y-intercepts per band for L8 TO L7 regression line - David Roy

  y = (oli.select(['B2','B3','B4','B5','B6','B7'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) # select OLI bands 2-7 and rename them to match L7 band names
             .subtract(itcp.multiply(10000)).divide(slopes)                                # ...multiply the y-intercept bands by 10000 to match the scale of the L7 bands then apply the line equation - subtract the intercept and divide by the slope
             .set('system:time_start', oli.get('system:time_start')))                     # ...set the output system:time_start metadata to the input image time_start otherwise it is null
  return y.toShort()                                                                       # return the image as short to match the type of the other data

# ------ FILTER A COLLECTION FUNCTION -----
def filterCollection(year, startDay, endDay, sensor, aoi):
  return (ee.ImageCollection('LANDSAT/'+ sensor + '/C01/T1_SR')
           .filterBounds(aoi)
           .filterDate(f'{year}-{startDay}', f'{year}-{endDay}'))

# ------ BUILD A COLLECTION FOR A GIVEN SENSOR AND YEAR -----
def buildSensorYearCollection(year, startDay, endDay, sensor, aoi):
  startMonth = int(startDay[0:2])
  endMonth = int(endDay[0:2])
  if(startMonth > endMonth):
    oldYear = str(int(year)-1)
    newYear = year
    oldYearStartDay = startDay
    oldYearEndDay = '12-31'
    newYearStartDay = '01-01'
    newYearEndDay = endDay
    
    oldYearCollection = filterCollection(oldYear, oldYearStartDay, oldYearEndDay, sensor, aoi)
    newYearCollection = filterCollection(newYear, newYearStartDay, newYearEndDay, sensor, aoi)
    
    srCollection = ee.ImageCollection(oldYearCollection.merge(newYearCollection))
  else:
    srCollection = filterCollection(year, startDay, endDay, sensor, aoi)
  
  
  return srCollection

# ------ RETRIEVE A SENSOR SR COLLECTION FUNCTION -----
def getSRcollection(year, startDay, endDay, sensor, aoi):
  
  #  get a landsat collection for given year, day range, and sensor
  srCollection = buildSensorYearCollection(year, startDay, endDay, sensor, aoi)
  srCollection = srCollection.map(maskLsr)

  # apply the harmonization function to LC08 (if LC08), subset bands, unmask, and resample           
  srCollection = (srCollection.map(lambda img: 
    ee.Image(
      ee.Algorithms.If(
        sensor == 'LC08',                                                  # condition - if image is OLI
        harmonizationRoy(img),                                    # true - then apply the L8 TO L7 alignment function after unmasking pixels that were previosuly masked (why/when are pixels masked)
        img.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])                   # false - else select out the reflectance bands from the non-OLI image
           .set('system:time_start', img.get('system:time_start'))         # ...set the output system:time_start metadata to the input image time_start otherwise it is null
      )
    )))
  return srCollection
  
# ------ FUNCTION TO COMBINE LT05, LE07, & LC08 COLLECTIONS -----
def getCombinedSRcollection(year, startDay, endDay, aoi):
    lt5 = getSRcollection(year, startDay, endDay, 'LT05', aoi)       # get TM collection for a given year, date range, and area
    le7 = getSRcollection(year, startDay, endDay, 'LE07', aoi)       # get ETM+ collection for a given year, date range, and area
    lc8 = getSRcollection(year, startDay, endDay, 'LC08', aoi)       # get OLI collection for a given year, date range, and area
    mergedCollection = ee.ImageCollection(lt5.merge(le7).merge(lc8)) # merge the individual sensor collections into one imageCollection object
    return mergedCollection                                              # return the Imagecollection
# ------ FUNCTION TO REDUCE COLLECTION TO SINGLE IMAGE PER YEAR BY MEDOID -----
#  make a medoid composite with equal weight among indices
def medoidMosaic(inCollection, dummyCollection):
  
  #fill in missing years with the dummy collection
  imageCount = inCollection.toList(1).length()                                                            #get the number of images 
  finalCollection = ee.ImageCollection(ee.Algorithms.If(imageCount.gt(0), inCollection, dummyCollection)) # if the number of images in this year is 0, then use the dummy collection, otherwise use the SR collection
  
  # calculate median across images in collection per band
  median = finalCollection.median()                                                                       # calculate the median of the annual image collection - returns a single 6 band image - the collection median per band
  
  # calculate the different between the median and the observation per image per band
  difFromMedian = (finalCollection.map(lambda img: ee.Image(img).subtract(median).pow(ee.Image.constant(2)).reduce('sum').addBands(img)
                                          # get the difference between each image/band and the corresponding band median and take to power of 2 to make negatives positive and make greater differences weight more
                                        #  per image in collection, sum the powered difference across the bands - set this as the first band add the SR bands to it - now a 7 band image collection
                                    ))
  
  # get the medoid by selecting the image pixel with the smallest difference between median and observation per band 
  return ee.ImageCollection(difFromMedian).reduce(ee.Reducer.min(7)).select([1,2,3,4,5,6], ['B1','B2','B3','B4','B5','B7']) # find the powered difference that is the least - what image object is the closest to the median of teh collection - and then subset the SR bands and name them - leave behind the powered difference band

#------ FUNCTION TO APPLY MEDOID COMPOSITING FUNCTION TO A COLLECTION -------------------------------------------
def buildMosaic(year, startDay, endDay, aoi, dummyCollection):  # create a temp variable to hold the upcoming annual mosiac
  collection = getCombinedSRcollection(year, startDay, endDay, aoi)  # get the SR collection
  img = (medoidMosaic(collection, dummyCollection)                     # apply the medoidMosaic function to reduce the collection to single image per year by medoid 
              .set('system:time_start', (ee.Date.fromYMD(ee.Number(year).toInt(),8,1))))  # add the year to each medoid image - the data is hard-coded Aug 1st 
  return ee.Image(img)                                                   # return as image object

# ------ FUNCTION TO BUILD ANNUAL MOSAIC COLLECTION ------------------------------
def buildSRcollection(startYear, endYear, startDay, endDay, aoi):
  dummyCollection = ee.ImageCollection([ee.Image([0,0,0,0,0,0]).mask(ee.Image(0))]) # make an image collection from an image with 6 bands all set to 0 and then make them masked values
  imgs = ee.ImageCollection([])                                                                         # create empty array to fill
  for i in range(startYear,endYear+1):                                      # for each year from hard defined start to end build medoid composite and then add to empty img array
    tmp = buildMosaic(i, startDay, endDay, aoi, dummyCollection)                    # build the median mosaic for a given year
    imgs = imgs.merge(ee.ImageCollection(tmp.set('system:time_start', (ee.Date.fromYMD(i,8,1))).set('year', i)))       # concatenate the annual image medoid to the collection (img) and set the date of the image - hard coded to the year that is being worked on for Aug 1st
  return imgs                                                       # return the array img array as an image collection

### Initiate timeframes and AOI

In [0]:
# Define the study period as the dry season (Jan-April) from 1988 to 2019
startYear = 1988
endYear = 2019
startDay = '01-01'
endDay = '04-30'
# AOI is the centroid of one Landsat tile, e.g. 
aoi = (ee.Feature(ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
            .filterDate('2019-01-01', '2019-01-31')
            .filterMetadata('WRS_PATH','equals',137) # Landsat Path
            .filterMetadata('WRS_ROW','equals',44) # Landsat Row
            .first().geometry()).centroid().geometry())
pprint ({'Centroid of the select path/row':aoi.getInfo()})

### Construct and display Landsat time series

In [0]:
allSensorsSR = buildSRcollection(startYear, endYear, startDay, endDay, aoi)
allSensorsSR = allSensorsSR.map(lambda img: img.updateMask(img))

In [0]:
pprint ({'Number of annual mediod composites': allSensorsSR.size().getInfo()})

In [0]:
pprint ({'First annual mediod composites': allSensorsSR.first().getInfo()})

In [0]:
# The image input data are the Landsat mediod composite of startYear and endYear.
image = allSensorsSR.filter(ee.Filter.eq('year',startYear)).first()
image1 = allSensorsSR.filter(ee.Filter.eq('year',endYear)).first()

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 3000})
mapIdDict1 = image1.getMapId({'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 3000})
map = folium.Map(location=[22.,89.], zoom_start=7)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Landsat Mediod Mosaic for {startYear}',
  ).add_to(map)
folium.TileLayer(
    tiles=mapIdDict1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Landsat Mediod Mosaic for {endYear}',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Enhanced Saliniy Indice Prep

>The ESI is based on an assumption that the reflectance values of
Landsat blue, red, and NIR bands are higher for salt-affected soils especially
when soil moisture is low (Khan and Sato, 2001; Nurmemet
et al., 2015). This concurs with several previous studies (Nurmemet
et al., 2015; Allbed et al., 2018, 2014). The ESI was employed to characterize
spatiotemporal variability of salinity in the study area. The ESI values
range between−1 and 1, and gives higher values for highly salinized soil
and vice versa.

> $ESI = (-EVI) = (2.5 *(NIR-R))/(NIR+6*R-7.5*B + 1)$

Reference: Tran, T. V., Tran, D. X., Myint, S. W., Huang, C. Y., Pham, H. V., Luu, T. H., & Vo, T. M. (2019). Examining spatiotemporal salinity dynamics in the Mekong River Delta using Landsat time series imagery and a spatial regression approach. Science of the total environment, 687, 1087-1097.

### Define function to calculate ESI

In [0]:
# Function to calculate the ESI
def calcESI(image):
  evi = (image.expression('2.5 * (b("B4")/10000 - b("B3")/10000) / (b("B4")/10000 + 6*b("B3")/10000 - 7.5*b("B1")/10000 + 1) * (-1)').toFloat().rename('ESI'))
  return evi.set({'system:time_start': image.get('system:time_start'),'year': image.get('year')})

In [0]:
# Calculate ESI for the time series
allSensorsSR= allSensorsSR.map(calcESI)

In [0]:
pprint ({'Check for ESI image':allSensorsSR_ESI.first().getInfo()})

Try to understand the ESI values for different land cover/use classes on the RGB image

In [0]:
# change the number to any value between 1988 and 2019, then the according year ESI image will be displayed below, the greener the image is, the high potential of salinized soil concurs
yearID = 2016

mapid = allSensorsSR_ESI.filter(ee.Filter.eq('year',yearID)).first().getMapId({'min': -0.8, 'max': 0.8, 'palette': ['fffdcd', 'e1cd73', 'aaac20', '5f920c', '187328', '144b2a', '172313']})
mapid1 = allSensorsSR.filter(ee.Filter.eq('year',yearID)).first().getMapId({'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 3000})
map = folium.Map(location=[22., 89.], zoom_start=7)
folium.TileLayer(
    tiles=mapid1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Landsat RGB {yearID}',
  ).add_to(map)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'ESI {yearID}',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Trend Analysis 1988-2019

The [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) measures how two continuous signals covary over time and indicate the linear relationship as a number between -1 (negatively correlated) to 0 (not correlated) to 1 (perfectly correlated). It is intuitive, easy to understand, and easy to interpret.

Here, the two continuous signals are **time** and **ESI time series**


### Prepare a time constant band

In [0]:
esiTrendCollection = (allSensorsSR_ESI
                          .map(lambda image: image.addBands(image.metadata('year').rename('constant'))))

In [0]:
pprint ({'Add a time constant band': esiTrendCollection.first().getInfo()})

### Caclulate Pearsons correlation coefficients
By applying `ee.Reducer.pearsonsCorrelation()` on the prepared time series, there are two bands being generated as an image output. ***Band 0 'correlation'*** countains the correlation coefficients ranging from -1 to +1, and ***Band 1 'p-value'*** contains two-side P-value indicating the signifcance of the trends.

In [0]:
reduced = esiTrendCollection.select(['constant', 'ESI']).reduce(ee.Reducer.pearsonsCorrelation())

In [0]:
pprint ({'PearsonsCorrelation Outputs': reduced.getInfo()})

### Map the significant trends only

To determine whether the correlation between variables is significant, compare the p-value to your significance level. Usually, a significance level (denoted as α or alpha) of 0.05 works well. An α of 0.05 indicates that the risk of concluding that a correlation exists—when, actually, no correlation exists—is 5%. The p-value tells you whether the correlation coefficient is significantly different from 0. (A coefficient of 0 indicates that there is no linear relationship.)
>P-value ≤ α: The correlation is statistically significant
If the p-value is less than or equal to the significance level, then you can conclude that the correlation is different from 0.

>P-value > α: The correlation is not statistically significant
If the p-value is greater than the significance level, then you cannot conclude that the correlation is different from 0.

In [0]:
sigreduced = reduced.select(0).multiply(reduced.select(1).lt(0.05)).rename('correlation')

sigreduced_format = (sigreduced.expression(
    "(b() > 0) ? 2" +
      ": (b() == 0) ? 3" +
        ": (b() < 0) ? 1" +
          ": 0"
))


In [0]:
mapid = sigreduced_format.updateMask(sigreduced_format.neq(0)).getMapId({'min': 1, 'max': 3, 'palette': ['green', 'orange', 'fffdcd']})
map = folium.Map(location=[22., 89.], zoom_start=7)
# Add custom basemaps
basemaps['Google Maps'].add_to(map)
basemaps['Google Satellite Hybrid'].add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Significant ESI Trends',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Export the product

In [0]:
task= ee.batch.Export.image.toDrive(
            image = sigreduced_format.updateMask(sigreduced_format.eq(2)).gt(0).toByte(),
            description = f'CTCN101_ESI_Trend',
            folder = f'CTCN101',
            region= sigreduced_format.geometry().getInfo()["coordinates"],    
            scale= 10,
            maxPixels= int(2e9),
            crs='EPSG:4326')
task.start()

# Block until the task completes.
print('Running image export to Google Drive...')
import time
while task.active():
  time.sleep(30)

# Error condition
if task.status()['state'] != 'COMPLETED':
  print('Error with image export.')
else:
  print('Image export completed.')